In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import csv



%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')

import pymc3 as pm
import theano.tensor as T
import theano

from scipy.stats import mode, chisquare

from sklearn.metrics import confusion_matrix, accuracy_score

import lasagne



In [2]:
# read flash.dat to a list of lists
datContent = [i.strip().split() for i in open("DES_full_wavelets.dat").readlines()]
datContent1 = [i.strip().split() for i in open("key.txt").readlines()]
columns=datContent[0]
columns1=datContent1[0]

In [3]:
features = pd.DataFrame(datContent[1:], columns=columns)
features=features.drop(['Object'], axis=1)
features.head()

,C0,C1,C2,C3,C4,C5,C6,C7,C8,C9,...,C12,C13,C14,C15,C16,C17,C18,C19,C20,z
0,-25.00037477444413,306.79211490489143,356.8997649727597,36.56806414373808,81.96294271832443,62.84881350054351,-149.50953399302256,49.388635007835,46.425898986402416,9.202334885218706,...,-33.037657156773214,-2.972694976441513,14.275091023477701,43.27783546040833,36.14978526004475,42.272794165931174,34.722632574076236,-1.0936417697512764,-35.21439165696788,0.3502
1,-238.79206891687346,155.3919979441455,211.36807494174155,-3.746101978293055,-93.19830815652585,42.334705719188065,-0.4838348067168141,130.21422038778377,-38.95153007406436,-21.025637471790123,...,73.82622017894873,65.57448246149349,-12.685975243515733,-6.071823311914481,8.494442815914777,-14.946762084511736,21.59773335882574,-11.420969334214146,1.7374948480963388,0.9609
2,-542.521162149438,70.30591950576184,83.55855291754986,-62.43710158940596,-4.303015725470706,13.865165481969552,-10.760797295484963,27.974491688346184,-2.373651075888283,17.846619786960538,...,6.706933251134956,11.655275670677133,5.120015670925272,-12.657090739176272,-3.9683953727566843,-4.338903653156308,16.50174347535103,10.526471774932752,-8.92679194932048,0.5628
3,-209.04457680773876,-121.1521579037945,59.6778141143199,22.282682854904284,-134.8065942940524,-18.231881130250255,-10.396626106209759,31.89273041660975,24.781259200927778,-25.653730436879304,...,10.092535994029824,2.115467645344072,69.00457558938463,28.33006929131822,-22.327955845622725,-14.938633312832923,6.709835775927774,-9.719449198277808,5.925270987983696,0.4747
4,-448.45014262661607,130.38673932244154,-59.986039979245646,60.42144611544627,-38.94952538238488,-29.11613660545993,39.45988907037251,-27.08410046683938,-23.877828822879913,17.637273025995523,...,33.72356414146945,-36.80300636482759,-3.5666978828339744,3.297053642332119,17.83264974893807,-41.45221511454384,-8.489533409627624,14.044875152417134,8.466453914603711,0.8882


In [4]:
labels = pd.DataFrame(datContent1[1:], columns=columns1)
labels=labels.drop(['Object'], axis=1)
labels.head()

,Type
0,2
1,2
2,2
3,2
4,3


In [5]:
msk = np.random.rand(len(features)) < 0.8

X_train = features[msk].values
y_train = features[msk].values
X_test = features[~msk].values
y_tset = features[~msk].values

In [6]:
input_var = theano.shared(X_train[:500,...].astype(np.float64))
target_var = theano.shared(y_train[:500,...].astype(np.float64))


In [7]:
def build_ann(init):
    l_in = lasagne.layers.InputLayer(shape=(1, 22),
                                     input_var=input_var)

    # Add a fully-connected layer of 800 units, using the linear rectifier, and
    # initializing weights with Glorot's scheme (which is the default anyway):
    n_hid1 = 22
    l_hid1 = lasagne.layers.DenseLayer(
        l_in, num_units=n_hid1,
        nonlinearity=lasagne.nonlinearities.tanh,
        b=init,
        W=init
    )

    n_hid2 = 22
    # Another 800-unit layer:
    l_hid2 = lasagne.layers.DenseLayer(
        l_hid1, num_units=n_hid2,
        nonlinearity=lasagne.nonlinearities.tanh,
        b=init,
        W=init
    )

    # Finally, we'll add the fully-connected output layer, of 10 softmax units:
    l_out = lasagne.layers.DenseLayer(
        l_hid2, num_units=10,
        nonlinearity=lasagne.nonlinearities.softmax,
        b=init,
        W=init
    )
    
    prediction = lasagne.layers.get_output(l_out)
    
    # 10 discrete output classes -> pymc3 categorical distribution
    out = pm.Categorical('out', 
                         prediction,
                         observed=target_var)
    
    return out


In [8]:


class GaussWeights(object):
    def __init__(self):
        self.count = 0
    def __call__(self, shape):
        self.count += 1
        return pm.Normal('w%d' % self.count, mu=0, sd=.1, 
                         testval=np.random.normal(size=shape).astype(np.float64),
                         shape=shape)



In [9]:
from six.moves import zip

# Tensors and RV that will be using mini-batches
minibatch_tensors = [input_var, target_var]

# Generator that returns mini-batches in each iteration
def create_minibatch(data, batchsize=500):
    
    rng = np.random.RandomState(0)
    start_idx = 0
    while True:
        # Return random data samples of set size batchsize each iteration
        ixs = rng.randint(data.shape[0], size=batchsize)
        yield data[ixs]

minibatches = zip(
    create_minibatch(X_train, 500),
    create_minibatch(y_train, 500),
)

total_size = len(y_train)

def run_advi(likelihood, advi_iters=50000):
    # Train on train data
    input_var.set_value(X_train[:500, ...])
    target_var.set_value(y_train[:500, ...])
    
    v_params = pm.variational.advi_minibatch(
        n=advi_iters, minibatch_tensors=minibatch_tensors, 
        minibatch_RVs=[likelihood], minibatches=minibatches, 
        total_size=total_size, learning_rate=1e-2, epsilon=1.0
    )
    trace = pm.variational.sample_vp(v_params, draws=500)
    
    # Predict on test data
    input_var.set_value(X_test)
    target_var.set_value(y_test)
    
    ppc = pm.sample_ppc(trace, samples=100)
    y_pred = mode(ppc['out'], axis=0).mode[0, :]
    
    return v_params, trace, ppc, y_pred


In [10]:
with pm.Model() as neural_network:
    likelihood = build_ann(GaussWeights())
    v_params, trace, ppc, y_pred = run_advi(likelihood)




RuntimeError: cannot initialize parameters: the provided callable did not return an array-like value

In [ ]:
print(pm.__version__)